| cod | provincia        |
|-----|------------------|
| 15  | A CORUÑA         |
| 03  | ALACANT/ALICANTE |
| 02 | ALBACETE |
| 04 | ALMERÍA |
| 01 | ARABA/ÁLAVA |
| 33 | ASTURIAS |
| 05 | ÁVILA |
| 06 | BADAJOZ |
| 08 | BARCELONA |
| 48 | BIZKAIA |
| 09 | BURGOS |
| 10 | CÁCERES |
| 11 | CÁDIZ |
| 39 | CANTABRIA |
| 12 | CASTELLÓ/CASTELLÓN |
| 51 | CEUTA |
| 13 | CIUDAD REAL |
| 14 | CÓRDOBA |
| 16 | CUENCA |
| 20 | GIPUZKOA |
| 17 | GIRONA |
| 18 | GRANADA |
| 19 | GUADALAJARA |
| 21 | HUELVA |
| 22 | HUESCA |
| 07 | ILLES BALEARS |
| 23 | JAÉN |
| 26 | LA RIOJA |
| 35 | LAS PALMAS |
| 24 | LEÓN |
| 25 | LLEIDA |
| 27 | LUGO |
| 28 | MADRID |
| 29 | MÁLAGA |
| 52 | MELILLA |
| 30 | MURCIA |
| 31 | NAVARRA |
| 32 | OURENSE |
| 34 | PALENCIA |
| 36 | PONTEVEDRA |
| 37 | SALAMANCA |
| 38 | SANTA CRUZ TENERIFE |
| 40 | SEGOVIA |
| 41 | SEVILLA |
| 42 | SORIA |
| 43 | TARRAGONA |
| 44 | TERUEL |
| 45 | TOLEDO |
| 46 | VALÈNCIA/VALENCIA |
| 47 | VALLADOLID |
| 49 | ZAMORA |
| 50 | ZARAGOZA |

In [101]:
from bs4 import BeautifulSoup

def get_info(content):
    soup = BeautifulSoup(content)
    
    tr = soup.find_all(attrs={'class': 'row100'})

    datos = []
    
    for elem in tr:
        td = elem.find_all('td')
        info = [x.find_all('div')[1].contents[0] for x in td[:5]]
        datos.append(dict(
            num = elem.find_all('span')[0].get('data-sec'),
            nombre = info[0],
            formato = info[1],
            anho = info[2],
            densidad = info[3],
            tamanho = info[4]
        ))

    return datos

In [103]:
import requests
import json
import math

url = 'https://centrodedescargas.cnig.es/CentroDescargas/checkAllFiles'

data = dict(
    codSerie = 'LIDA2',
    codAgr = 'MOMDT',
    codTipoArchivo = 'LAZ',
    codProvincia = '37', # Introducir código de la provincia que se quiere descargar
    numPagina = 1
)

r = requests.post(url, data)
numFiles = json.loads(r.content)['totalFiles']

pages = math.ceil(numFiles / 20)
print(pages)

220


In [105]:
import requests

datos = []

for i in range(1, pages + 1):
    print(i)
    url = 'https://centrodedescargas.cnig.es/CentroDescargas/archivosSerie'
    
    r = requests.post(url=url, data={
        'codSerie': 'LIDA2',
        'codAgr': 'MOMDT',
        'codTipoArchivo': 'LAZ',
        'codProvincia': '37', 
        'numPagina': i
    })
    
    datos += get_info(r.content)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import box

df = pd.DataFrame(datos)

df['coord_x'] = df.apply(lambda x: int(x.nombre.split('-')[4]) * 1000, axis=1)
df['coord_y'] = df.apply(lambda x: int(x.nombre.split('-')[5]) * 1000, axis=1)

df_25830 = df[df.coord_x < 600_000]

gdf_25830 = gpd.GeoDataFrame(df_25830, geometry=box(df_25830.coord_x, df_25830.coord_y - 2000, df_25830.coord_x + 2000, df_25830.coord_y), crs='epsg:25830') 

In [153]:
df_25829 = df[df.coord_x > 600_000]

gdf_25829 = gpd.GeoDataFrame(df_25829, geometry=box(df_25829.coord_x, df_25829.coord_y - 2000, df_25829.coord_x + 2000, df_25829.coord_y), crs='epsg:25829') 

In [154]:
gdf_25829.to_file('FINALES/SALAMANCA_25829')

In [151]:
gdf_25830.to_file('FINALES/SALAMANCA_25830')